# QM9 Dataset
[reference examples](https://github.com/deepchem/deepchem/tree/master/examples/qm9)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import tensorflow as tf
import sklearn 
import deepchem as dc

In [ ]:
import pathlib
model_dir = pathlib.Path('../models')

**Multitask**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
def mtr_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = mtr_model_builder(params, model_dir)

print("training MTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Graph Convolution**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
def gc_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = gc_model_builder(params, model_dir)

print("training GC model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating GC model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Multitask** _(Coulomb Matrix Featurizer)_  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm9/qm9_tf_models.py)

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
from dc.models import MultiTaskFitTransformRegressor

def mtftr_model_builder(model_params, model_dir):
    return MultitaskFitTransformRegressor(model_dir, **model_params)

In [ ]:
fit_transformers = [
    dc.trans.CoulombFitTransformer(train_dataset)
]

metric = [
    dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks),
    "n_features": [29, 29],
    "weight_init_stddevs": [1.0/np.sqrt(400), 1.0/np.sqrt(100), 1.0/np.sqrt(100)],
    "bias_init_consts": [0.0, 0.0, 0.0]
    "layer_sizes": [400, 100, 100],
    "dropouts": [0.01, 0.01, 0.01],
    "fit_transformers": fit_transformers,
    "batch_size": 32,
    "learning_rate": 0.001,
    "momentum": 0.8,
    "n_evals": 10
}

model = mtftr_model_builder(params, model_dir)

print("training MTFTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTFTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Deep Tensor Neural Network**  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm9/qm9_DTNN.py)

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
def dtnn_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = dtnn_model_builder(params, model_dir)

print("training DTNN model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating DTNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Message Passing Neural Network**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='random') # default
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
def mpnn_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = mpnn_model_builder(params, model_dir)

print("training MPNN model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MPNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)